In [ ]:
%reload_ext autoreload
%autoreload 2

# Wikipedia

In [1]:
from coleta.corretor_paginas_wikipedia import CorretorEstruturaCategorias
from coleta.wikipedia import CategoriaCollector, PaginaCollector, WikipediaDownloader
from urllib.parse import urlencode
from pymongo import MongoClient
import time
import pandas as pd

wikipedia_downloader = WikipediaDownloader()
pagina_collector = PaginaCollector(collection_paginas='pages_brasil', collection_paginas_conteudo='pages_content_brasil')
categoria_collector = CategoriaCollector(collection_categorias='categories_brasil', pagina_collector=pagina_collector)
country = 'Brasil'

## Definir categoria raiz

Em nosso caso de estudo a categoria raiz sera Geografia do Brasil. Abaixo esta o documento correspondente, obtido atraves de procura feita 
no resultado da consulta de category_members da categoria "Categoria:Brasil".

In [ ]:
is_adicionar_categoria_raiz = False
if is_adicionar_categoria_raiz:
    categoria_geografia_brasil = {'pageid': 15122,
    'ns': 14,
    'title': 'Categoria:Geografia do Brasil',
    'type': 'subcat', 
    'country': 'Brasil'}
    categoria_collector.add_categories_for_check([categoria_geografia_brasil])
# category_members = wikipedia_downloader.get_category_members(cmtitle='Categoria:Brasil')

## Iteração de download de categorias

- Categories status
    * Check: waiting for someone to check if it has to be downloaded
    * Waiting: waiting for dowload
    * Skeep: skeep download of this category
    * Done: downloaded

### Passo 1 - Procurar categorias candidatas a download

In [ ]:
categorias_candidatas_para_download = categoria_collector.get_categories_for_check()

In [ ]:
for ccpd in categorias_candidatas_para_download[:10]:
    print(f'{ccpd["pageid"]} - {ccpd["title"]}')

### Passo 2 - Selecionar para download e skeep

In [ ]:
categorias_selecionadas_para_download = []
if len(categorias_selecionadas_para_download) > 0:
    categoria_collector.set_categories_for_download(categorias_selecionadas_para_download)

In [ ]:
categorias_selecionadas_para_skeep = []
if len(categorias_selecionadas_para_skeep) > 0:
    categoria_collector.set_categories_for_skeep(categorias_selecionadas_para_skeep)

### Passo 3 - Fazer download de selecionadas

In [ ]:
categorias_para_download = categoria_collector.get_categories_for_download()
print(f'{len(categorias_para_download)} categorias recuperadas para download...')
for cpd in categorias_para_download:
    categoria_collector.download_category_tree(cpd, cpd['country'])

## Corrigir estrutura de categorias

- Deve ser executado após o download de todas as categorias_para_download
- É necessário porque  várias paginas sao referenciadas por mais de uma categoria e por isso sao inseridas duas vezes por cada categoria. Para evitar de 
ficar verificando se uma pagina foi incluida ou nao no download da categoria, preferi fazer o donwload mais de uma vez e corrigir tudo de uma vez apos o download.

In [ ]:
# corretor_estrutura_categorias = CorretorEstruturaCategorias(collection_name_incorreto='', collection_correto='')

## Download de páginas (wikitext)

- Pages status (wikitext)
    * Waiting: waiting for download
    * Done: Download done

In [ ]:
# Executar apenas uma vez
# pagina_collector.set_pages_for_wikitext_download()

In [ ]:
paginas_download_wikitext = pagina_collector.get_pages_for_download()

In [ ]:
pagina_collector.run_wikitext_download(5)

## Download de páginas (text)

- Pages status (text)
    * Waiting: waiting for download
    * Donw: Download done

In [2]:
# Executar apenas uma vez
# pagina_collector.set_pages_for_text_download()

In [3]:
pagina_collector.run_text_download(5)

Running text download for 5 pages
Page text [406](Geografia do Brasil) downloaded successfuly (1 of 5)
Page text [1753565](Atlas do Desenvolvimento Humano do Brasil) downloaded successfuly (2 of 5)
Page text [3958949](Cachoeira da Pirapora) downloaded successfuly (3 of 5)
Page text [6062483](Carta Geográfica do Brasil (1922)) downloaded successfuly (4 of 5)
Page text [3302088](Colar metropolitano) downloaded successfuly (5 of 5)


6

## Backup do que já foi feito download

In [ ]:
run_backup = False
if run_backup:
    categories = list(mongo_categories.find())
    categories_df = pd.DataFrame(categories)
    categories_df.to_csv('categories.csv')

    pages = list(mongo_pages.find())
    pages_df = pd.DataFrame(pages)
    pages_df.to_csv('pages.csv')

### Verificar categorias e páginas sem country

In [ ]:
# Pages
pages_no_country = list(db.pages.find({"country": {"$exists": False }}))
print(f'{len(pages_no_country)} pages found with no country')

# Categories
categories_no_country = list(db.categories.find({"country": {"$exists": False }}))
print(f'{len(categories_no_country)} categories found with no country')

In [ ]:
def show_download_statistics():
    pdc = get_pages_downloads_counts()
    print(f'Page download counts: {pdc}')
    pages_wikitext_counts = mongo_content.count_documents({})
    print(f'Pages with wikitext: {pages_wikitext_counts}')
# show_download_statistics()

### Download page texts

In [ ]:
run_text_download(303)
show_text_download_statistics()

In [ ]:
pcs = list(mongo_content.aggregate(
    [ 
        { '$group': { '_id': '$pageid', 'total': { '$sum': 1 } } }
    ]
))

In [ ]:
dobrado = [p for p in pcs if p['total'] > 1]


In [ ]:
def find_distinct_pages():
    return list(mongo_pages.aggregate(
        [ 
            { '$group': { '_id': '$pageid', 'total': { '$sum': 1 } } },
            { '$sort': { 'total': -1 }}
        ]
    ))

def count_distinct_pages():
    return len(find_distinct_pages())

print(f'We have downloaded {count_distinct_pages()} distinct pages')

In [ ]:
pages_dobrado = [p for p in pcs if p['total'] > 1]
pages_dobrado[0]

## Correção collection pages

- Problema: a mesma página tinha sido baixada mais de uma vez porque era referenciada por várias coleções
- Solução
    * Renomear collection pages para pages_incorreto (feito no mongo shell)
    * Recriar a collection pages com estrutura correta
        * A categoria agora será um objeto com id e título ao invés de estes valores estarem soltos na páginas. Assim ficará melhor estuturado
        * O objeto da página terá uma collection de categorias. Cada página lida da estrutura antiga terá na nova collection um array com as categorias que a referenciam
    * Renomear pages_content para pages_content incorreto

In [ ]:
from pymongo import MongoClient

mongo_client = MongoClient()
db = mongo_client.wikipedia
collection_incorreto = db['pages_brasil_incorreto']
collection_correto = db['pages_brasil']

In [ ]:
# Obtem todas paginas de collection incorreta
todas_paginas_incorreto = list(collection_incorreto.find({}))

In [ ]:
def criar_objetos_categoria(todas_paginas_incorreto):
    # Corrigir categoria das paginas incorretas. Cria objeto para categoria e coloca id e titulo
    for pagina_incorreto in todas_paginas_incorreto:
        category = {
            'category_id': pagina_incorreto['category_id'],
            'category_title': pagina_incorreto['category_title']
        }
        pagina_incorreto['category'] = category
        del pagina_incorreto['category_id']
        del pagina_incorreto['category_title']
    return todas_paginas_incorreto

todas_paginas_incorreto = criar_objetos_categoria(todas_paginas_incorreto)

In [ ]:
def agregar_categorias_por_pagina(todas_paginas_incorreto):
    # Dictionario que mapeia uma pageid para documento da pagina
    agregado_paginas_incorreto = {}

    for pagina_incorreto in todas_paginas_incorreto:
        pageid_incorreto = pagina_incorreto['pageid']
        if pageid_incorreto in agregado_paginas_incorreto:
            existente = agregado_paginas_incorreto[pageid_incorreto]
            existente['categories'].append(pagina_incorreto['category'])
        else:
            pagina_incorreto['categories'] = [pagina_incorreto['category']]
            agregado_paginas_incorreto[pageid_incorreto] = pagina_incorreto
        del pagina_incorreto['category']
    
    return agregado_paginas_incorreto

agregado_paginas_incorreto = agregar_categorias_por_pagina(todas_paginas_incorreto)

In [ ]:
# Insere as páginas corrigidas no mongo
paginas_agregadas = list(agregado_paginas_incorreto.values())
collection_correto.insert_many(paginas_agregadas)

In [ ]:
# Conta páginas para ver o resultado
print(collection_correto.count_documents({}))
print(collection_incorreto.count_documents({}))

## Análise documentos download

In [ ]:
import seaborn as sns
import pandas as pd

### Análise wikitext

In [ ]:
pcs = list(mongo_content.find({}))

In [ ]:
lengths_wikitext = [len(pc['wikitext']) for pc in pcs]
df = pd.DataFrame(lengths_wikitext, columns=['Length'])
df.head()

In [ ]:
sns.displot(df, x='Length', bins=20)

In [ ]:
# Verificando se todas as páginas srealmente têm conteúdo
df['Length'].describe()

## Fix downloaded text

- It was stored in the documents the whole structure of the answer
- It should store only the ext

In [ ]:
pcs = list(mongo_content.find({}))
for pc in pcs:
    mongo_content.update_one(
        { 'pageid': pc['pageid'] },
        { '$set': { 'text': pc['text']['parse']['text'] } }
    )

### Análise text

In [ ]:
pcs = list(mongo_content.find({}))

In [ ]:
lengths_text = [len(pc['text']) for pc in pcs]
df = pd.DataFrame(lengths_text, columns=['Length'])
df.head()

In [ ]:
sns.displot(df, x='Length', bins=20)

In [ ]:
# Verificando se todas as páginas srealmente têm conteúdo
df['Length'].describe()

In [5]:
from repository import nerds_viajantes as nerds_viajantes_repo
import pandas as pd

In [2]:
p = nerds_viajantes_repo.read_published()

In [6]:
pd.DataFrame(nerds_viajantes_repo.execute_query(nerds_viajantes_repo.PUBLISHED_POSTS_QUERY))

,0,1,2,3,4,5,6
0,725,2011-11-10 19:23:52,<strong>O Blog - Nerds Viajantes</strong>\r\n\...,Quem Somos,quem-somos,2016-06-23 10:35:41,0
1,11,2012-01-21 12:00:01,"No nosso primeiro dia em <a href=""www.nerdsvia...",Santiago - Museu de Arte Pré-Colombiana Fechad...,santiago-museu-de-arte-pre-colombiana-fechado-...,2013-04-19 16:41:23,1
2,29,2012-01-20 09:57:45,"O TripAdvisor publicou uma <a href=""http://www...",TripAdvisor - Top 25 Hotéis,tripadvisor-top-25-hoteis,2012-12-11 09:15:21,0
3,60,2012-02-24 12:16:22,"Sexta-feira em <a href=""http://www.nerdsviajan...",Santiago - Restaurante Aquí Está Coco,santiago-restaurante-aqui-esta-coco,2013-04-19 18:13:46,5
4,69,2012-01-26 07:53:50,Viajar para a Índia é pode ser um sonho para u...,Índia - Folha divulga dicas de sobrevivência,india-folha-dicas-de-sobrevivencia,2012-10-04 21:53:27,0
...,...,...,...,...,...,...,...
1024,15244,2018-05-11 17:55:02,Já está disponível o <strong>Papel de Parede d...,Papel de Parede de Maio de 2018,papel-de-parede-de-maio-de-2018,2018-05-11 17:55:02,0
1025,15269,2018-06-12 16:15:34,O <strong>Papel de Parede de Junho de 2018</st...,Papel de Parede de Junho de 2018,papel-de-parede-de-junho-de-2018,2018-07-04 19:36:38,0
1026,15324,2018-08-25 07:07:24,Disponibilizamos para nossos leitores o <stron...,Papel de Parede de Agosto de 2018,papel-de-parede-de-agosto-de-2018,2018-09-28 19:47:14,0
1027,15353,2018-09-29 11:05:50,Antes tarde do que nunca! Disponibilizamos par...,Papel de Parede de Setembro de 2018,papel-de-parede-de-setembro-de-2018,2019-09-01 10:20:29,0
